In [ ]:
import json
import os
import tweepy as tw
import pandas as pd
import time
from datetime import date, datetime
import pymongo


from secrets import consumer_key, consumer_secret, access_token, access_token_secret

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# Create connection variable
conn = 'mongodb://localhost:27017'

# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)

# Connect to a database. Will create one if not already available.
db = client.tweets

## Search tweets by hashtag

In [ ]:
# tweets_ht = tw.Cursor(api.search, 
#                       q ="$SPX",
#                       lang = 'en', 
#                       since="2018-06-29",
#                       until = "2018-06-29").items(5)
# tweets_ht

In [ ]:
accounts = ['@LizAnnSonders', '@morganhousel', '@EconguyRosie','@CitronResearch', '@TruthGundlach', '@AswathDamodaran']
screen_names = ['LizAnnSonders', 'morganhousel', 'EconguyRosie', 'CitronResearch', 'TruthGundlach', 'AswathDamodaran']

# accounts = ['@cnbc', '@nytimesbusiness', '@WSJMarkets', '@stlouisfed', '@jimcramer', '@TheStalwart', '@ReformedBroker']
# screen_names = ['CNBC','nytimesbusiness', 'WSJmarkets', 'stlouisfed', 'jimcramer', 'TheStalwart', 'ReformedBroker']

In [ ]:
def max_tweet_id(name):
    return db['capstone_finance'].find_one({'screen_name':name}, sort = [('unix_timestamp', 1)])['tweet_id']

In [ ]:
def account_tweets(account, name, count = 5):
    
    try:
        max_id = max_tweet_id(name)
        tweets = api.user_timeline(screen_name=account, max_id = max_id,
                           count=count, tweet_mode='extended')
    except: 
        tweets = api.user_timeline(screen_name=account,
                           count=count, tweet_mode='extended')
    
    try:
    
#         tweets = api.user_timeline(screen_name=account, max_id = max_id,
#                                    count=count, tweet_mode='extended')

        tweets_by_account = {}

        for tweet in tweets:
            tweet_dict = {}
            tweet_dict['username'] = tweet.user.name
            tweet_dict['follower_count'] = tweet.user.followers_count
            tweet_dict['screen_name'] = tweet.user.screen_name
            tweet_dict['text'] = tweet.full_text
            tweet_dict['tweet_id'] =tweet.id
            tweet_dict['retweet_count'] = tweet.retweet_count
            tweet_dict['unix_timestamp'] = time.mktime(tweet.created_at.timetuple()) 
            tweet_dict['datetime'] = tweet.created_at.strftime("%d-%b-%Y %H:%M:%S.%f")
            tweet_dict['favorite_count'] = tweet.favorite_count
            db.capstone_finance.update_one({'tweet_id': tweet.id}, {"$set": tweet_dict}, upsert = True)
            
    except:
        print("==========================================")
        print("Something went wrong:")
        print(account, name)

In [ ]:
time.sleep(900)
for i in range(20):
    print('Waiting for 15min')
    time.sleep(900)
    print('Proceed to the next batch')
    for account, name in zip(accounts, screen_names):
        account_tweets(account, name, count = 3200)
        

In [ ]:
# Query tweets with right keywords, figure out which accounts have tweets 
# covering the entire year

In [ ]:
# mongodb pipeline:
# {"text": {$in:[/arket/, /SPX/, /SPY/, /stock/, /S&P/, /conom/, /Fed/]},
# , unix_timestamp: {$gte: 1530401041}}

# {"text": {$in:[/arket/, /SPX/, /SPY/, /stock/, /S&P/, /conom/, /Fed/, /bull/, /bear/,/momentum/, /voltility/,/treasury/,/Powell/,/policy/, /spending/, /tax/, /tariff/, /hike/, /trade/]}, unix_timestamp: {$gte: 1530401041}}